In [1]:
import numpy as np
import pandas as pd
import sys
from numpy.linalg import *
from numpy import dot
import matplotlib.pyplot as plt
from multiprocessing import Process

In [2]:
train = pd.read_csv('X_train.csv', header=None, names=['X','Y'])
t = pd.read_csv('T_train.csv', header=None, names=['H'])
train['target'] = t['H']
train = train.sample(frac=1)
test_set = pd.read_csv('X_test.csv', header=None, names=['X','Y'])
#validation, train = np.split(csv.sample(frac=1), [len(csv)//5])

In [3]:
# validation = validation[:200]
# train = train[:1000]

In [4]:
class Gaussian2D():
    
    def __init__(self, mean_x, mean_y, variance_x=1, variance_y=1):
        self.mean_x = mean_x
        self.mean_y = mean_y
        self.variance_x = variance_x
        self.variance_y = variance_y
        self.prob = None
        
    def probability(self, x, y):
        self.prob = (np.exp(-((x-self.mean_x)**2)/(2*self.variance_x**2) - 
                             ((y-self.mean_y)**2)/(2*self.variance_y**2)))
        return self.prob
    
    def __str__(self):
        return 'Gaussian2D with mean {}, {}; variance {}, {}'.format(self.mean_x, self.mean_y, self.variance_x, self.variance_y)

In [5]:
class utility():
    
    def data_to_graph(self, data):
        graph = np.zeros((1081, 1081))
        M = data['target'].max()
        for index, row in data.iterrows():
            graph[row['X']-1, row['Y']-1] = row['target']
        return graph
    
    def find_peaks(self, graph, window_size_x=20, window_size_y=20, put=1):
        # put: 一個window放幾個basis
        peaks = []
        for i in range(0, graph.shape[1]-graph.shape[1]%window_size_x, window_size_x):
            fat_column = graph[:, i:i+window_size_x]
            for j in range(0, graph.shape[0]-graph.shape[0]%window_size_y, window_size_y):
                # 每個window放兩個, 如果最大值是0, 就只放一個隨機點  --- 改成放在中間
                # 在training時使用後面的flag, e.g. random / peak
                window = fat_column[j:j+window_size_y, :].flatten()
                #print(np.max(window))
                if np.max(window) == 0:
                    peaks.append((j+window_size_y//2, i + window_size_x//2, 'random'))
                    #peaks.append((np.random.randint(j, j+window_size_y), np.random.randint(i, i+window_size_x), 'random'))
                    #continue
                elif np.max(window) is None:
                    continue
                else:
                    peak = np.where(window == max(window))[0]
                    peaks.append((j+int(peak[0])//window_size_x, i+int(peak[0])%window_size_x, 'peak'))
                # 一個window可能有兩個以上的peak, 只取兩個, 未滿兩個以random補  ---- 不這樣做了, 每格只放一個
#                 peak = np.where(window == max(window))[0]
#                 #peaks.append((j+int(peak[0])//window_size_x, i+int(peak[0])%window_size_x, 'peak'))
#                 if len(peak) > 1:
#                     peaks.append((j+int(peak[0])//window_size_x, i+int(peak[0])%window_size_x, 'peak'))
#                     #peaks.append((j+int(peak[1])//window_size_x, i+int(peak[1])%window_size_x, 'peak'))
#                 # window裡面最高的是0, 隨便放一個就好
#                 else:
#                     peak = (j+int(peak[0])//window_size_x, i+int(peak[0])%window_size_x, 'peak')
#                     peaks.append(peak)
# #                     while True:
# #                         r = (np.random.randint(j, j+window_size_y), np.random.randint(i, i+window_size_x), 'random')
# #                         if r[0] == peak[0] and r[1] == peak[1]:
# #                             continue
# #                         else:
# #                             peaks.append(r)
# #                             break
        return peaks
    
    # implement of k_fold spliter, 封裝方式感覺不好, 有機會改一下
    def k_fold_spliter(self, data, k=5, return_indicies=False):
        #data = data.sample(frac=1)
        train_val_pair = [] # data sets of each fold
        indicies = [] # index of (train, val)
        for i in range(0, len(data), len(data)//k):
            if i + len(data)//k >= len(data):
                val = data.iloc[i:]
                val_index = list(range(i, len(data)))
                train = data.iloc[:i]
                train_index = list(range(i))
                indicies.append((train_index, val_index))
                train_val_pair.append((train, val))
            else:
                val = data.iloc[i:i+len(data)//k]
                val_index = list(range(i, i+len(data)//k))
                train = data.iloc[:i].append(data.iloc[i+len(data)//k:])
                train_index = list(range(i))
                train_index.extend(list(range(i+len(data)//k, len(data))))
                indicies.append((train_index, val_index))
                train_val_pair.append((train, val))
        if return_indicies:     
            return(train_val_pair, indicies)
        else:
            return train_val_pair

In [6]:
class bayesian_linear_regression(utility):
    
    def __init__(self):
        self.feature_basis = None
        self.design_matrix = None
        self.W_ML = None
        self.W_MAP = None
        self.m_N = None
        self.S_N_inv = None
        self.predictive_distribution_function = None

    def create_predictive_distribution_function(self, input_data, target, alpha, beta):
        print('compute S_N_inv')
        S_N_inv = alpha * np.eye(len(self.feature_basis)) + beta * dot(self.design_matrix.T, self.design_matrix)
        # 為了方便, 將m0設為zero
        print('compute m_N')
        m_N = beta * solve(S_N_inv, dot(self.design_matrix.T, target))
        self.S_N_inv = S_N_inv
        self.m_N = m_N
        def func(x, S_N_inv, m_N, design_matrix, beta=beta):
            mu = dot(m_N.T, design_matrix)
            #s_N_square = 1.0/beta + dot(design_matrix.T, solve(S_N_inv, design_matrix))
            #return (mu, s_N_square)
            return mu
        self.predictive_distribution_function = func
        
    # 1. create feature basis
    def create_grid_gaussian_basis(self, var_x=100, var_y=100, grid_size_land=70, grid_size_sea=100, heuristic=True, show=False):
        # multiple regression
        # 座標點值1~1081, 切grid
        # 假設都由高斯分布組成
        # mean為坐標點, var隨意設
        feature_basis = [1] # 1: bias
        for i in range(200, 1082, grid_size_land):
            for j in range(1, 1082, grid_size_land):
                feature_basis.append(Gaussian2D(i, j, var_x, var_y))
        for i in range(1, 201, grid_size_sea):
            for j in range(1, 1082, grid_size_sea):
                if heuristic:
                    feature_basis.append(Gaussian2D(i, j, 2*var_x, 2*var_y))
                else:
                    feature_basis.append(Gaussian2D(i, j, var_x, var_y))
        self.feature_basis = feature_basis
        if show:
            return self.feature_basis        

    def test_overfit_basis(self, train, var_x=100, var_y=100, show=False):# 每個座標放高斯
        feature_basis = [1]
        for index, row in train.iterrows():
            feature_basis.append(Gaussian2D(row['X'], row['Y'], var_x, var_y))
        self.feature_basis = feature_basis
        if show:
            return feature_basis
    
    def create_peak_gaussian_basis(self, train, p_var_x=5, p_var_y=5, r_var_x=20, r_var_y=20, window_size_x=None, window_size_y=None, show=False):
        # 設計方式如pdf
        graph = self.data_to_graph(train)
        if window_size_x is None:
            window_size_x = 20
        if window_size_y is None:
            window_size_y = 20
        peaks = self.find_peaks(graph, window_size_x, window_size_y)
        feature_basis = [1] # 1: bias
        for peak in peaks:
            if peak[2] == 'peak':
                # 此處不consistence, create basis時使用x,y; 但寫peak時寫成row,col... 有空改
                feature_basis.append(Gaussian2D(peak[1], peak[0], p_var_x, p_var_y))
            else:
                feature_basis.append(Gaussian2D(peak[1], peak[0], r_var_x, r_var_y))
        self.feature_basis = feature_basis
        if show:
            return self.feature_basis
    
    # 2. create design matrix
    def create_design_matrix(self, data, change=True, return_result=False, show=False, n_workers=None, verbose=True):
        print('create design matrix...')
        design_matrix = np.zeros((len(data), len(self.feature_basis)))
        design_matrix[:,0] = 1
        num_features = design_matrix.shape[1]
        if n_workers is None:
            for i in range(1, num_features): # by column, 0th is basis, skip
                design_matrix[:,i] = data.apply(lambda row: self.feature_basis[i].probability(row['X'], row['Y']), axis=1)
                if verbose:
#                     done = 80 * i // num_features
#                     sys.stdout.write("\r[%s%s]" % ('▶' * done, '•' * (80-done)))
                    done = 100 * i // num_features
                    sys.stdout.write('\r complete {}%'.format(done))
            print()
        else:
            pass
            result_queue = multiprocessing.Queue()
            design_matricies = []
            # 以feature數去平行處理
            for i in range(1, num_features, num_features//n_workers):
                if i + num_features//n_workers >= num_features:
                    design_matricies.append()
        if change:
            self.design_matrix = design_matrix
        if show:
            print(self.design_matrix)
        if return_result:
            return design_matrix
    
    # 3_1. create W_ML
    def fit_with_W_ML(self, input_data, target, k_fold=None, show=False, get_WLs=False):
        self.create_design_matrix(input_data)
        print('compute W_ML...')
        self.W_ML = dot(pinv(self.design_matrix), target)
        
#         if k_fold is None:
#             self.create_design_matrix(input_data)
#             print('compute W_ML...')
#             self.W_ML = dot(pinv(self.design_matrix), target)
            
#         else:
#             W_MLs = []
#             data = input_data.copy()
#             data['target'] = target.values
#             count = 0
#             design_matrix = self.create_design_matrix(input_data, False, True)
#             k_fold_pairs = self.k_fold_spliter(input_data, 5, return_indicies=True)
#             for i in range(k_fold):
#                 train = k_fold_pairs[0][i][0]
#                 val = k_fold_pairs[0][i][1]
#                 train_index = k_fold_pairs[1][i][0]
#                 #print(train_index)
#                 val_index = k_fold_pairs[1][i][1]
#                 #print(val_index)
#                 count += 1
#                 print('compute W_ML {}/{} ...'.format(count, k_fold))
#                 W_ML = dot(pinv(design_matrix[train_index,:]), target.iloc[train_index])
#                 W_MLs.append(W_ML)
#                 print('train fold', i, self.evaluate_W_ML(train, target.iloc[train_index], W_ML))
#                 print('val fold', i, self.evaluate_W_ML(val, target.iloc[val_index], W_ML))
#             W = np.zeros(W_MLs[0].shape)
#             for W_ML in W_MLs:
#                 W += W_ML
#             self.W_ML = W / k_fold
        
        if show:
            return self.W_ML
        
    # 3_2. create W_MAP
    def fit_with_W_MAP(self, input_data=None, target=None, k_fold=None, show=False, s=20, beta=1000000, reg=0.1):
        if self.design_matrix is None:
            self.create_design_matrix(input_data)
        if k_fold is None:
            self.W_MAP = reg*np.eye(len(self.feature_basis)) + dot(self.design_matrix.T, self.design_matrix)
            self.W_MAP = dot(inv(self.W_MAP), dot(self.design_matrix.T, target))
#             t_tilta = target
#             t_tilta *= np.sqrt(beta)
#             t_tilta = np.append(t_tilta, [0]*len(self.feature_basis))

#             design_matrix_tilta = self.design_matrix[train_index, :]
#             design_matrix_tilta *= np.sqrt(beta)
#             design_matrix_tilta = np.append(design_matrix_tilta, s*np.eye(len(self.feature_basis)), axis=0)

#             q, r = np.linalg.qr(design_matrix_tilta)
#             W_MAP = dot(dot(inv(r), q.T), t_tilta)
#             self.W_MAP = W_MAP
#         q, r = np.linalg.qr(self.design_matrix)
#         self.W_MAP = dot(dot(inv(r), q.T), target)
        else:
            if type(beta) != list:
                betas = range(1000000, 20000001, 1000000)
            else:
                betas = beta
            data = input_data.copy()
            data['target'] = target.values
            k_fold_pairs = self.k_fold_spliter(input_data, k_fold, return_indicies=True)
            best_beta = None
            print(betas)
            for beta in betas:
                W_MAPs = []
                count = 0
                print('beta {}'.format(beta))
                train_err = 0
                val_err = 0
                #reg = reg / 5
                for i in range(k_fold):
                    train = k_fold_pairs[0][i][0]
                    val = k_fold_pairs[0][i][1]
                    train_index = k_fold_pairs[1][i][0]
                    #print(train_index)
                    val_index = k_fold_pairs[1][i][1]
                    #print(val_index)
                    count += 1
                    print('compute W_MAP {}/{} ...'.format(count, k_fold))
                    t_tilta = target.iloc[train_index]
                    t_tilta *= np.sqrt(beta)
                    t_tilta = np.append(t_tilta, [0]*len(self.feature_basis))
                    
                    design_matrix_tilta = self.design_matrix[train_index, :]
                    design_matrix_tilta *= np.sqrt(beta)
                    design_matrix_tilta = np.append(design_matrix_tilta, s*np.eye(len(self.feature_basis)), axis=0)

                    
                    q, r = np.linalg.qr(design_matrix_tilta)
                    W_MAP = dot(dot(inv(r), q.T), t_tilta)
                    #W_MAP = reg*np.eye(len(self.feature_basis)) + dot(self.design_matrix.T, self.design_matrix)
                    #W_MAP = dot(inv(W_MAP), dot(self.design_matrix.T, target))
                    tre = self.evaluate_W_MAP(train, target.iloc[train_index], W_MAP)
                    print('train fold', i, tre)
                    train_err += tre
                    vae = self.evaluate_W_MAP(val, target.iloc[val_index], W_MAP)
                    print('val fold', i, vae)
                    val_err += vae
                train_err /= k_fold
                val_err /= k_fold
                print('overall err of beta {}: {}'.format(beta, val_err))
                if best_beta is None:
                    best_beta = (val_err, beta, W_MAP)
                elif val_err < best_beta[0]:
                    best_beta = (val_err, beta, W_MAP)
            print('best beta is', best_beta[1])
            self.W_MAP = best_beta[2]
            self.beta = best_beta[1]
                
        if show:
            return self.W_MAP
    
    # 3_3. create predictive distribution model
    def fit_with_predictive_distribution_function(self, input_data, target, alpha=50, beta=10):
        if self.design_matrix is None:
            self.create_design_matrix(input_data)
        print('creating predictive distribution model')
        self.create_predictive_distribution_function(input_data, target, alpha, beta)
        
    # 4. predict new data
    def predict_with_W_ML(self, x, y, W_ML=None):
        if W_ML is None:
            W_ML = self.W_ML
        predict = W_ML[0] * self.feature_basis[0]
        for i in range(1, len(W_ML)):
            predict += W_ML[i] * self.feature_basis[i].probability(x, y)
        return predict
    
    def predict_with_W_MAP(self, x, y, W_MAP=None):
        if W_MAP is None:
            W_MAP = self.W_MAP
        predict = W_MAP[0] * self.feature_basis[0]
        for i in range(1, len(W_MAP)):
            predict += W_MAP[i] * self.feature_basis[i].probability(x, y)
        return predict

    def predict_with_predictive_distribution_function(self, x, y):
        phi_X = [1]
        phi_X.extend([feature.probability(x, y) for feature in self.feature_basis[1:]])
        phi_X = np.array(phi_X).reshape(len(self.feature_basis),1)
        predict = dot(self.m_N.T, phi_X)
        return predict[0]
    

    # 5. evaluate model
    def evaluate_W_ML(self, test_data_input, target, W_ML=None):
        print('evaluating with W_ML...')
        if W_ML is None:
            W_ML = self.W_ML
        x = np.array(test_data_input)
        t = np.array(target)
        err = 0
        for i in range(len(x)):
            predict = self.predict_with_W_ML(x[i,0], x[i,1], W_ML=W_ML)
            err += (predict - t[i])**2
        err /= len(x)
        return err
    
    def evaluate_W_MAP(self, test_data_input, target, W_MAP=None):
        print('evaluating with W_MAP...')
        if W_MAP is None:
            W_MAP = self.W_MAP
        x = np.array(test_data_input)
        t = np.array(target)
        err = 0
        for i in range(len(x)):
            predict = self.predict_with_W_MAP(x[i,0], x[i,1], W_MAP=W_MAP)
            err += (predict - t[i])**2
        err /= len(x)
        return err
    
    def evaluate_predictive_distribution_function(self, test_data_input, target):
        print('evaluating with predictive distribution function...')
        x = np.array(test_data_input)
        t = np.array(target)
        err = 0
        for i in range(len(x)):
            predict = self.predict_with_predictive_distribution_function(x[i,0], x[i,1])
            err += (predict - t[i])**2
        err /= len(x)
        return err

In [7]:
# 用於比較各種參數
def compare_train_validation(csv, i=1, j=81, k=2, compare='variance', get_models=False, v=None):
    models = []
    val, train = np.split(csv.sample(frac=1), [len(csv)//5])
    train_errs = []
    val_errs = []
    # 比較variance
    if compare == 'variance':
        for v in range(i, j, k):
            np.random.seed(1)
            clf = bayesian_linear_regression()
            # just a heuristic, 海平面應該比較平緩, 陸地的variance較低
            # grid default 20x20
            clf.create_peak_gaussian_basis(train, v, v, 70, 70)
            clf.fit_with_W_ML(train[['X','Y']], train['target'])
            models.append(clf)
            train_err = clf.evaluate_W_ML(train[['X', 'Y']], train['target'])
            train_errs.append(train_err)
            val_err = clf.evaluate_W_ML(val[['X','Y']], val['target'])
            val_errs.append(val_err)
            print('train err of {}: {}'.format(v, train_err))
            print('val err of {}: {}'.format(v, val_err))
        a, = plt.plot(range(i, j, k), np.log(train_errs), 'ro', label="train error")
        b, = plt.plot(range(i, j, k), np.log(val_errs), 'b^', label="validation error")
        plt.legend(handles=[a, b])
        plt.title('compare variance with grid size 20')
        plt.xlabel('variance')
        plt.ylabel('log_error')
        plt.savefig('train_val_variance_err_differ.png')
        plt.show()
        plt.close()
    
    # 比較grid size
    elif compare == 'gridsize':
        if v is None:
            v = 50
        for grid_size in range(i, j, k):
            np.random.seed(1)
            clf = bayesian_linear_regression()
            clf.create_peak_gaussian_basis(train, v, v, 70, 70, grid_size, grid_size)
            clf.fit_with_W_ML(train[['X','Y']], train['target'])
            models.append(clf)
            train_err = clf.evaluate_W_ML(train[['X', 'Y']], train['target'])
            train_errs.append(train_err)
            val_err = clf.evaluate_W_ML(val[['X','Y']], val['target'])
            val_errs.append(val_err)
            print('train err of {}: {}'.format(grid_size, train_err))
            print('val err of {}: {}'.format(grid_size, val_err))
        plt.style.use('ggplot')
        a, = plt.plot(range(i, j, k), np.log(train_errs), 'ro', label="train error")
        b, = plt.plot(range(i, j, k), np.log(val_errs), 'b^', label="validation error")
        plt.legend(handles=[a, b])
        plt.title('compare grid size with variance {}'.format(v))
        plt.xlabel('grid_size')
        plt.ylabel('log_error')
        plt.savefig('train_val_grid_err_differ.png')
        plt.show()
        plt.close()
        
    # 比較lambda
    elif compare == 'reg':
        if v is None:
            v = 20
        grid_size = 25
        for reg in range(i, j, k):
            np.random.seed(1)
            clf = bayesian_linear_regression()
            clf.create_peak_gaussian_basis(train, v, v, 70, 70, grid_size, grid_size)
            clf.fit_with_W_MAP(train[['X','Y']], train['target'], reg=reg/5)
            models.append(clf)
            train_err = clf.evaluate_W_MAP(train[['X', 'Y']], train['target'])
            train_errs.append(train_err)
            val_err = clf.evaluate_W_MAP(val[['X','Y']], val['target'])
            val_errs.append(val_err)
            print('train err of {}: {}'.format(grid_size, train_err))
            print('val err of {}: {}'.format(grid_size, val_err))
        plt.style.use('ggplot')
        a, = plt.plot(range(i, j, k), np.log(train_errs), 'ro', label="train error")
        b, = plt.plot(range(i, j, k), np.log(val_errs), 'b^', label="validation error")
        plt.legend(handles=[a, b])
        plt.title('compare reg differ with variance {}'.format(v))
        plt.xlabel('reg')
        plt.ylabel('log_error')
        plt.savefig('train_val_reg_err_differ.png')
        plt.show()
        plt.close()
    if get_models:
        return models

In [8]:
compare_train_validation(train[:500], 10, 80, 10) #比variance

create design matrix...
 complete 99%
compute W_ML...
evaluating with W_ML...
evaluating with W_ML...
train err of 10: 1.7108035189237509e-22
val err of 10: 377323.05850975006
create design matrix...
 complete 37%

KeyboardInterrupt: 

In [11]:
compare_train_validation(train[:500], 80, 10, -10, compare='gridsize') #比gridsize

create design matrix...
 complete 99%
compute W_ML...
evaluating with W_ML...
evaluating with W_ML...
train err of 80: 804.0849048182375
val err of 80: 3167.3169644701948
create design matrix...
 complete 81%

KeyboardInterrupt: 

In [ ]:
#compare_train_validation(train, 10, 80, 10, compare='reg') #比lambda

In [12]:
# ml
v = 20
t = train[:1000]
val, tr = np.split(t, [len(t)//5])
clf3 = bayesian_linear_regression()
clf3.create_peak_gaussian_basis(tr, v, v, 70, 70, 25, 25)
clf3.fit_with_W_ML(tr[['X','Y']], tr['target'])
train_err = clf3.evaluate_W_ML(tr[['X','Y']], tr['target'])
val_err = clf3.evaluate_W_ML(val[['X', 'Y']], val['target'])
print(train_err, val_err)
lst = np.array([clf3.predict_with_W_ML(row['X'], row['Y']) for index, row in test_set.iterrows()])
test_set['ML_target'] = lst
ML = test_set['ML_target']
ML.to_csv('ML.csv', index=False)

create design matrix...
 complete 99%
compute W_ML...
evaluating with W_ML...
evaluating with W_ML...
1.45874953671e-16 105040689223.0


KeyboardInterrupt: 

In [13]:
# MAP
v = 20
t = train[:1000]
val, tr = np.split(t, [len(t)//5])
clf3 = bayesian_linear_regression()
clf3.create_peak_gaussian_basis(tr, v, v, 70, 70, 25, 25)
clf3.fit_with_W_MAP(tr[['X','Y']], tr['target'], reg=0.1)
train_err = clf3.evaluate_W_MAP(tr[['X','Y']], tr['target'])
val_err = clf3.evaluate_W_MAP(val[['X', 'Y']], val['target'])
print(train_err, val_err)
lst = np.array([clf3.predict_with_W_MAP(row['X'], row['Y']) for index, row in test_set.iterrows()])
#test_set['MAP_target'] = lst
#MAP = test_set['MAP_target']
#MAP.to_csv('MAP1.csv', index=False)

create design matrix...
 complete 99%
evaluating with W_MAP...
evaluating with W_MAP...
469.700476234 1545.15889382


In [14]:
# cross validation with W_MAP for BETA
t = train[:500]
val, tr = np.split(t, [len(t)//5])
clf2 = bayesian_linear_regression()
clf2.create_peak_gaussian_basis(tr, 20, 20, 70, 70, 25, 25)
clf2.fit_with_W_MAP(tr[['X','Y']], tr['target'], 5)
train_err = clf2.evaluate_W_MAP(tr[['X','Y']], tr['target'])
val_err = clf2.evaluate_W_MAP(val[['X', 'Y']], val['target'])
print(train_err, val_err)
lst = np.array([clf2.predict_with_W_MAP(row['X'], row['Y']) for index, row in test_set.iterrows()])
test_set['MAP_target'] = lst
MAP = test_set['MAP_target']
MAP.to_csv('MAP.csv', index=False)

create design matrix...
 complete 99%
range(1000000, 20000001, 1000000)
beta 1000000
compute W_MAP 1/5 ...
evaluating with W_MAP...
train fold 0 21.4115121294
evaluating with W_MAP...
val fold 0 11458.3394641
compute W_MAP 2/5 ...
evaluating with W_MAP...
train fold 1 33.1279911246
evaluating with W_MAP...
val fold 1 12165.0162185
compute W_MAP 3/5 ...
evaluating with W_MAP...
train fold 2 45.0720503319
evaluating with W_MAP...
val fold 2 15096.5289616
compute W_MAP 4/5 ...
evaluating with W_MAP...
train fold 3 26.3624129399
evaluating with W_MAP...


KeyboardInterrupt: 

In [15]:
# predictive dist
clf1 = bayesian_linear_regression()
t = train[:500]
val, tr = np.split(t, [len(t)//5])
v = 20
clf1.create_peak_gaussian_basis(tr, v, v, 70, 70, 25, 25)
clf1.fit_with_predictive_distribution_function(tr[['X','Y']], tr['target'])
train_err = clf1.evaluate_predictive_distribution_function(tr[['X','Y']], tr['target'])
val_err = clf1.evaluate_predictive_distribution_function(val[['X', 'Y']], val['target'])
print(train_err, val_err)
lst = np.array([clf1.predict_with_predictive_distribution_function(row['X'], row['Y']) for index, row in test_set.iterrows()])
test_set['bayesian_target'] = lst
bayes = test_set['bayesian_target']
bayes.to_csv('Bayesian.csv', index=False)

create design matrix...
 complete 99%
creating predictive distribution model
compute S_N_inv
compute m_N
evaluating with predictive distribution function...
evaluating with predictive distribution function...
1371.64250714 1698.51654673
